In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np
%matplotlib inline
from sklearn.externals import joblib
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
import os
from sklearn.ensemble import VotingClassifier


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
df = pd.read_pickle("../dataset.pkl")

In [3]:
df.columns

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  'G11',            'G12',            'G13',            'G14',
                  'G15',            'G16',            'G17',            'G18',
                  'G19',            'G20'],
      dtype='object', length=503)

In [4]:
x = df.iloc[:,:483]
x.shape
#483 = 3(adm_days,gender,age)+480(lab values)

(58976, 483)

In [5]:
y = df.iloc[:,483:]


In [6]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split
seed=243 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state = seed)

In [8]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler(feature_range=(0, 1))
x_train_mm = mm.fit_transform(x_train)
x_test_mm = mm.transform(x_test)

In [12]:
from sklearn.naive_bayes import MultinomialNB
nb = OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))  



In [14]:
#cat = joblib.load("./cat1-C.pkl")
#xgb = joblib.load("./XGB1-C.pkl")
lgbm_str = joblib.load("./LGBM1-C.pkl")
#rf = joblib.load("./Randomforest-C.pkl")
#mod = joblib.load("XGB1-C.pkl") lol

In [18]:
nb.fit(x_train_mm, y_train)

y_pred_mm = nb.predict(x_test_mm)
print(jaccard_score(y_test,y_pred_mm,average='micro'))
print(jaccard_score(y_test,y_pred_mm,average='macro'))
print(roc_auc_score(y_test,y_pred_mm))
print(hamming_loss(y_test,y_pred_mm))

joblib.dump(nb,'NB-struc.pkl')

0.3054572296129779
0.11771688483166527
0.5155449841693456
0.2339832711653668


['NB-struc.pkl']

In [ ]:
#joblib.dump(, ".pkl")
# can we see corr bw lgbm and this? go ahead if u know how to..ill check lol looks wierd 

In [48]:

estimators=[ ('lgbm', lgbm), ('nb', nb), ('xgb', xgb)]#yo 1 sec its mostly gonna be lgbm winning, think abt it ,add rf
ensemble = OneVsRestClassifier(VotingClassifier(estimators, voting='soft')) # i just started off with this
ensemble.fit(x_train_mm, y_train)
#print(ensemble.score(x_test_mm, y_test))
# ya obv, but thats not done na..it is. ok ohh ya chill i got this

OneVsRestClassifier(estimator=VotingClassifier(estimators=[('lgbm',
                                                            OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                                         class_weight={0: 0.286,
                                                                                                                       1: 0.714},
                                                                                                         colsample_bytree=1.0,
                                                                                                         importance_type='split',
                                                                                                         iterations=1500,
                                                                                                         learning_rate=0.15,
                                          

In [49]:
#from sklearn.model_selection import cross_val_predict,cross_val_score
y_pred = ensemble.predict(x_test_mm)

In [50]:

ensemble_auc = roc_auc_score(y_test,y_pred)
ensemble_ham = hamming_loss(y_test,y_pred)
print(ensemble_auc,ensemble_ham)

0.6588478823601948 0.16976941336046117


In [51]:
ensemble_soft = OneVsRestClassifier(VotingClassifier(estimators, voting='soft')) # i just started off with this
ensemble_soft.fit(x_train_mm, y_train)
y_pred = ensemble_soft.predict(x_test_mm)
ensemble_auc = roc_auc_score(y_test,y_pred)
ensemble_ham = hamming_loss(y_test,y_pred)
print(ensemble_auc,ensemble_ham)

KeyboardInterrupt: 

In [54]:
nice_estimators=[ ('lgbm', lgbm), ('rf',rf), ('xgb', xgb), ('cat',cat)]
ensemble_prob = OneVsRestClassifier(VotingClassifier(nice_estimators, voting='soft')) # i just started off with this
ensemble_prob.fit(x_train,y_train)
y_pred_prob = ensemble_prob.predict(x_test)
auc = roc_auc_score(y_test,y_pred_prob)
ham = hamming_loss(y_test,y_pred_prob)
print(auc,ham)
#yo 1 sec its mostly gonna be lgbm winning, think abt it ,add rf
# add




'''
rf - 0.6367671964034114n
xgb - 0.6643581037392547
lgbm - 0.6900906965008473   remember this!!!! hmm
cat - 0.691337534365617
nb - 0.5155449841693456

hamming_loss
rf - 0.17626879168079576
xgb - 0.16991070419351192
lgbm - 0.20071775743189782
cat - 0.19957612750084774
nb - 0.2339832711653668

'''


Learning rate set to 0.035347


0:	learn: 0.6623759	total: 177ms	remaining: 4m 25s
1:	learn: 0.6341898	total: 187ms	remaining: 2m 20s
2:	learn: 0.6090685	total: 197ms	remaining: 1m 38s
3:	learn: 0.5862992	total: 207ms	remaining: 1m 17s
4:	learn: 0.5658497	total: 217ms	remaining: 1m 5s
5:	learn: 0.5445775	total: 227ms	remaining: 56.5s
6:	learn: 0.5248830	total: 248ms	remaining: 52.9s
7:	learn: 0.5086486	total: 306ms	remaining: 57s
8:	learn: 0.4945988	total: 518ms	remaining: 1m 25s
9:	learn: 0.4815318	total: 609ms	remaining: 1m 30s
10:	learn: 0.4685941	total: 618ms	remaining: 1m 23s
11:	learn: 0.4576486	total: 627ms	remaining: 1m 17s
12:	learn: 0.4466809	total: 637ms	remaining: 1m 12s
13:	learn: 0.4374155	total: 650ms	remaining: 1m 9s
14:	learn: 0.4281234	total: 660ms	remaining: 1m 5s
15:	learn: 0.4197792	total: 670ms	remaining: 1m 2s
16:	learn: 0.4120451	total: 679ms	remaining: 59.2s
17:	learn: 0.4053152	total: 688ms	remaining: 56.7s
18:	learn: 0.3984435	total: 698ms	remaining: 54.4s
19:	learn: 0.3925584	total: 707ms	

167:	learn: 0.2898195	total: 3.09s	remaining: 24.5s
168:	learn: 0.2897237	total: 3.1s	remaining: 24.4s
169:	learn: 0.2896822	total: 3.11s	remaining: 24.3s
170:	learn: 0.2896118	total: 3.13s	remaining: 24.3s
171:	learn: 0.2895246	total: 3.14s	remaining: 24.2s
172:	learn: 0.2893547	total: 3.15s	remaining: 24.1s
173:	learn: 0.2892403	total: 3.16s	remaining: 24.1s
174:	learn: 0.2891726	total: 3.17s	remaining: 24s
175:	learn: 0.2891155	total: 3.18s	remaining: 23.9s
176:	learn: 0.2889194	total: 3.19s	remaining: 23.8s
177:	learn: 0.2888564	total: 3.2s	remaining: 23.7s
178:	learn: 0.2888260	total: 3.36s	remaining: 24.8s
179:	learn: 0.2887526	total: 3.55s	remaining: 26s
180:	learn: 0.2886808	total: 3.55s	remaining: 25.9s
181:	learn: 0.2886104	total: 3.56s	remaining: 25.8s
182:	learn: 0.2884777	total: 3.57s	remaining: 25.7s
183:	learn: 0.2884408	total: 3.58s	remaining: 25.6s
184:	learn: 0.2883282	total: 3.59s	remaining: 25.5s
185:	learn: 0.2882482	total: 3.6s	remaining: 25.4s
186:	learn: 0.28811

329:	learn: 0.2769343	total: 5.98s	remaining: 21.2s
330:	learn: 0.2768616	total: 6.21s	remaining: 21.9s
331:	learn: 0.2767284	total: 6.21s	remaining: 21.9s
332:	learn: 0.2766623	total: 6.22s	remaining: 21.8s
333:	learn: 0.2765817	total: 6.23s	remaining: 21.8s
334:	learn: 0.2764998	total: 6.24s	remaining: 21.7s
335:	learn: 0.2764459	total: 6.25s	remaining: 21.6s
336:	learn: 0.2763654	total: 6.26s	remaining: 21.6s
337:	learn: 0.2763189	total: 6.26s	remaining: 21.5s
338:	learn: 0.2762581	total: 6.27s	remaining: 21.5s
339:	learn: 0.2761268	total: 6.28s	remaining: 21.4s
340:	learn: 0.2760708	total: 6.29s	remaining: 21.4s
341:	learn: 0.2760125	total: 6.3s	remaining: 21.3s
342:	learn: 0.2759143	total: 6.3s	remaining: 21.3s
343:	learn: 0.2758115	total: 6.31s	remaining: 21.2s
344:	learn: 0.2757236	total: 6.32s	remaining: 21.2s
345:	learn: 0.2756019	total: 6.33s	remaining: 21.1s
346:	learn: 0.2754868	total: 6.34s	remaining: 21.1s
347:	learn: 0.2754373	total: 6.35s	remaining: 21s
348:	learn: 0.27

502:	learn: 0.2663274	total: 8.7s	remaining: 17.3s
503:	learn: 0.2662962	total: 8.71s	remaining: 17.2s
504:	learn: 0.2662227	total: 8.72s	remaining: 17.2s
505:	learn: 0.2661821	total: 8.73s	remaining: 17.1s
506:	learn: 0.2661280	total: 8.74s	remaining: 17.1s
507:	learn: 0.2660754	total: 8.75s	remaining: 17.1s
508:	learn: 0.2660032	total: 8.75s	remaining: 17s
509:	learn: 0.2659582	total: 8.76s	remaining: 17s
510:	learn: 0.2659329	total: 8.77s	remaining: 17s
511:	learn: 0.2659071	total: 8.78s	remaining: 16.9s
512:	learn: 0.2658540	total: 8.79s	remaining: 16.9s
513:	learn: 0.2658042	total: 8.87s	remaining: 17s
514:	learn: 0.2657274	total: 9.13s	remaining: 17.5s
515:	learn: 0.2656376	total: 9.14s	remaining: 17.4s
516:	learn: 0.2656002	total: 9.15s	remaining: 17.4s
517:	learn: 0.2655801	total: 9.16s	remaining: 17.4s
518:	learn: 0.2655477	total: 9.17s	remaining: 17.3s
519:	learn: 0.2654835	total: 9.18s	remaining: 17.3s
520:	learn: 0.2654321	total: 9.18s	remaining: 17.3s
521:	learn: 0.2654069

...

856:	learn: 0.3958304	total: 16.4s	remaining: 12.3s
857:	learn: 0.3957940	total: 16.4s	remaining: 12.3s
858:	learn: 0.3957682	total: 16.4s	remaining: 12.3s
859:	learn: 0.3957357	total: 16.5s	remaining: 12.2s
860:	learn: 0.3956884	total: 16.5s	remaining: 12.2s
861:	learn: 0.3956484	total: 16.6s	remaining: 12.3s
862:	learn: 0.3956034	total: 16.6s	remaining: 12.3s
863:	learn: 0.3955798	total: 16.6s	remaining: 12.2s
864:	learn: 0.3955431	total: 16.6s	remaining: 12.2s
865:	learn: 0.3954963	total: 16.8s	remaining: 12.3s
866:	learn: 0.3954521	total: 17s	remaining: 12.4s
867:	learn: 0.3954227	total: 17s	remaining: 12.4s
868:	learn: 0.3953840	total: 17s	remaining: 12.3s
869:	learn: 0.3953491	total: 17s	remaining: 12.3s
870:	learn: 0.3953400	total: 17s	remaining: 12.3s
871:	learn: 0.3952992	total: 17s	remaining: 12.3s
872:	learn: 0.3952734	total: 17s	remaining: 12.2s
873:	learn: 0.3952434	total: 17s	remaining: 12.2s
874:	learn: 0.3952111	total: 17s	remaining: 12.2s
875:	learn: 0.3951848	total: 1

1025:	learn: 0.3904297	total: 19.8s	remaining: 9.13s
1026:	learn: 0.3904089	total: 19.8s	remaining: 9.1s
1027:	learn: 0.3903888	total: 19.8s	remaining: 9.08s
1028:	learn: 0.3903446	total: 19.8s	remaining: 9.05s
1029:	learn: 0.3903122	total: 19.8s	remaining: 9.03s
1030:	learn: 0.3903039	total: 19.8s	remaining: 9.01s
1031:	learn: 0.3902988	total: 19.8s	remaining: 8.98s
1032:	learn: 0.3902876	total: 19.8s	remaining: 8.96s
1033:	learn: 0.3902416	total: 19.8s	remaining: 8.94s
1034:	learn: 0.3901985	total: 19.8s	remaining: 8.91s
1035:	learn: 0.3901682	total: 19.8s	remaining: 8.89s
1036:	learn: 0.3901289	total: 19.9s	remaining: 8.86s
1037:	learn: 0.3900969	total: 19.9s	remaining: 8.84s
1038:	learn: 0.3900756	total: 19.9s	remaining: 8.82s
1039:	learn: 0.3900538	total: 19.9s	remaining: 8.79s
1040:	learn: 0.3900180	total: 19.9s	remaining: 8.77s
1041:	learn: 0.3899627	total: 19.9s	remaining: 8.75s
1042:	learn: 0.3899092	total: 19.9s	remaining: 8.72s
1043:	learn: 0.3898827	total: 20s	remaining: 8.

1192:	learn: 0.3854956	total: 22.7s	remaining: 5.83s
1193:	learn: 0.3854696	total: 22.7s	remaining: 5.83s
1194:	learn: 0.3854385	total: 22.8s	remaining: 5.81s
1195:	learn: 0.3854152	total: 22.8s	remaining: 5.79s
1196:	learn: 0.3853857	total: 22.8s	remaining: 5.76s
1197:	learn: 0.3853696	total: 22.8s	remaining: 5.74s
1198:	learn: 0.3853456	total: 22.9s	remaining: 5.75s
1199:	learn: 0.3853255	total: 23.1s	remaining: 5.78s
1200:	learn: 0.3853167	total: 23.1s	remaining: 5.76s
1201:	learn: 0.3852827	total: 23.2s	remaining: 5.74s
1202:	learn: 0.3852425	total: 23.2s	remaining: 5.72s
1203:	learn: 0.3852352	total: 23.2s	remaining: 5.7s
1204:	learn: 0.3851981	total: 23.2s	remaining: 5.67s
1205:	learn: 0.3851796	total: 23.2s	remaining: 5.65s
1206:	learn: 0.3851605	total: 23.2s	remaining: 5.63s
1207:	learn: 0.3851381	total: 23.2s	remaining: 5.61s
1208:	learn: 0.3851105	total: 23.2s	remaining: 5.59s
1209:	learn: 0.3850798	total: 23.2s	remaining: 5.57s
1210:	learn: 0.3850517	total: 23.2s	remaining: 

1351:	learn: 0.3811315	total: 25.9s	remaining: 2.83s
1352:	learn: 0.3811131	total: 25.9s	remaining: 2.81s
1353:	learn: 0.3810897	total: 25.9s	remaining: 2.79s
1354:	learn: 0.3810614	total: 25.9s	remaining: 2.77s
1355:	learn: 0.3810369	total: 25.9s	remaining: 2.75s
1356:	learn: 0.3809911	total: 25.9s	remaining: 2.73s
1357:	learn: 0.3809559	total: 25.9s	remaining: 2.71s
1358:	learn: 0.3809203	total: 25.9s	remaining: 2.69s
1359:	learn: 0.3808817	total: 25.9s	remaining: 2.67s
1360:	learn: 0.3808591	total: 25.9s	remaining: 2.65s
1361:	learn: 0.3808168	total: 26s	remaining: 2.63s
1362:	learn: 0.3807995	total: 26s	remaining: 2.61s
1363:	learn: 0.3807450	total: 26s	remaining: 2.59s
1364:	learn: 0.3807270	total: 26.1s	remaining: 2.58s
1365:	learn: 0.3806954	total: 26.1s	remaining: 2.56s
1366:	learn: 0.3806654	total: 26.1s	remaining: 2.54s
1367:	learn: 0.3806537	total: 26.1s	remaining: 2.52s
1368:	learn: 0.3806339	total: 26.3s	remaining: 2.51s
1369:	learn: 0.3806012	total: 26.5s	remaining: 2.51s

'\nrf - 0.6367671964034114n\nxgb - 0.6643581037392547\nlgbm - 0.6900906965008473\ncat - 0.691337534365617\nnb - 0.5155449841693456\n\nhamming_loss\nrf - 0.17626879168079576\nxgb - 0.16991070419351192\nlgbm - 0.20071775743189782\ncat - 0.19957612750084774\nnb - 0.2339832711653668\n\n'

In [35]:
y_pred_rf = rf.predict_proba(x_train)
y_pred_xgb = xgb.predict_proba(x_train)
y_pred_lgbm = lgbm.predict_proba(x_train)
y_pred_cat = cat.predict_proba(x_train)
#np.hstack((A, B)

In [36]:
max_val_auc = 0.6367671964034114+0.6643581037392547+ 0.6900906965008473+0.691337534365617
max_val_ham = 0.17626879168079576+0.16991070419351192+0.20071775743189782+0.19957612750084774

y_pred_auc = 0.6367671964034114/max_val_auc*y_pred_rf + 0.6643581037392547/max_val_auc*y_pred_xgb+0.6900906965008473/max_val_auc*y_pred_lgbm+0.691337534365617/max_val_auc*y_pred_cat
y_pred_ham = 0.17626879168079576/max_val_ham*y_pred_rf + 0.16991070419351192/max_val_ham*y_pred_xgb+0.20071775743189782/max_val_ham*y_pred_lgbm+0.19957612750084774/max_val_ham*y_pred_cat
x_train = np.hstack((x_train,y_pred_auc))
x_train = np.hstack((x_train,y_pred_ham))
#y_pred_auc = y_pred_auc.round()
#y_pred_ham = y_pred_ham.round()


In [28]:
y_pred_rf1= rf.predict_proba(x_test)
y_pred_xgb1 = xgb.predict_proba(x_test)
y_pred_lgbm1 = lgbm.predict_proba(x_test)
y_pred_cat1 = cat.predict_proba(x_test)
max_val_auc = 0.6367671964034114+0.6643581037392547+ 0.6900906965008473+0.691337534365617
max_val_ham = 0.17626879168079576+0.16991070419351192+0.20071775743189782+0.19957612750084774

y_pred_auc1 = 0.6367671964034114/max_val_auc*y_pred_rf1 + 0.6643581037392547/max_val_auc*y_pred_xgb1+0.6900906965008473/max_val_auc*y_pred_lgbm1+0.691337534365617/max_val_auc*y_pred_cat1
y_pred_ham1 = 0.17626879168079576/max_val_ham*y_pred_rf1 + 0.16991070419351192/max_val_ham*y_pred_xgb1+0.20071775743189782/max_val_ham*y_pred_lgbm1+0.19957612750084774/max_val_ham*y_pred_cat1
x_test = np.hstack((x_test,y_pred_auc1))
x_test = np.hstack((x_test,y_pred_ham1))

In [29]:
x_train.shape#isnt 20 more missing...oh shoot got it ma bad

(50129, 523)

In [30]:
x_test.shape

(8847, 523)

In [31]:
#what classifier u wanna use? hmm this is the second block na,nope this is the meta model ohh rf
from sklearn.ensemble import AdaBoostClassifier
ada = OneVsRestClassifier(AdaBoostClassifier(n_estimators=1000,learning_rate=0.1)) 
ada.fit(x_train,y_train)
y_pred = ada.predict(x_test) 

auc = roc_auc_score(y_test,y_pred)
ham = hamming_loss(y_test,y_pred)
print(auc,ham)

0.6787713287624801 0.16783655476432688


In [32]:
auc = roc_auc_score(y_test,y_pred_auc)
ham = hamming_loss(y_test,y_pred_auc)
print(auc,ham)

ValueError: Found input variables with inconsistent numbers of samples: [8847, 50129]

In [1]:
auc = roc_auc_score(y_test,y_pred_ham)
ham = hamming_loss(y_test,y_pred_ham)
print(auc,ham)

NameError: name 'roc_auc_score' is not defined

from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingCVClassifier

sclf = OneVsRestClassifier(StackingCVClassifier(classifiers=[xgb,lgbm],
                            use_probas=True,
                            meta_classifier=cat,
                            random_state=42))
sclf.fit(x_train,y_train)
y_pred = sclf.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)

In [ ]:
#this bottom part for later...

sclf1 = OneVsRestClassifier(StackingCVClassifier(classifiers=[sclf,cat],
                            use_probas=True,
                            meta_classifier=rf,
                            random_state=42))

sclf1.fit(x_train,y_train)
y_pred = sclf1.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print(auc)